<a href="https://colab.research.google.com/github/Lalith321/BlenderAutoAnim/blob/master/3rd_Experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install tensorflow-gpu

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import os,sys,time
import sklearn as sk
from IPython import display
import matplotlib.pyplot as plt
#from sklearn.preprocessing import train_test_split
from sklearn.datasets import load_files
#from tensorflow_core._api.v2.compat.v1.random.experimental import Generator

In [0]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [0]:
# Linking google drive
from google.colab import drive
drive.mount('/content/drive')

In [0]:
def preprocess1(data):
    global X1
    data = tf.reshape(data,[300,268,1])
    data = tf.image.resize(data,[48,2304])
    data = tf.reshape(data,[1,48,48,48]) # Reshaping the data
    X1 = np.append(X1,data,axis = 0)
    return X1

In [0]:
root = "/content/drive/My Drive"
#TrainData = load_files("D:\\projects\\BlenderAutoAnimator\\Train")
TrainData = "/content/drive/My Drive/Train1"
filenames = []

X1 = np.array([],dtype=np.int64).reshape(0,48,48,48)
X2 = []

for target in os.listdir(TrainData):
    print(target)
    for f in os.listdir(os.path.join(TrainData+"/"+target)):
        o_data = (np.loadtxt(TrainData+"/"+target+"/"+f))
        o_data = o_data.reshape(int(o_data.shape[0]/67),67,4) # Turn the image to (300, 67, 4)
        initial = o_data.shape[0]
        if(initial<=300):
            data = o_data
            additionals = int((300-initial)%initial) # If image.shape[1] is less than 300 then add the extra data starting from frame 1 (10's digit)
            for i in range(int((300-initial)/initial)):
                data = np.vstack((data,o_data)) # additional daata (100's digit)
            data = np.vstack((data,o_data[:additionals])) # Create a bunch of data by stacking it one on the other
            # Special steps for resizing 3D data
            X1 = preprocess1(data)

In [0]:
print(X1.shape)

In [0]:
def preprocess2(preprocessing_input):
    global max_,min_
    preprocessing_input = np.asarray(preprocessing_input)
    preprocessing_input = preprocessing_input.reshape(preprocessing_input.shape[0], 48, 48, 48, 1).astype('float32')
    max_ = tf.math.reduce_max(preprocessing_input)
    min_ = tf.math.reduce_min(preprocessing_input)
    preprocessing_input = (preprocessing_input - min_) / (max_-min_) # Normalize the images to [-1, 1]
    return preprocessing_input
"""for element in train_dataset:
    print(element.shape[0])
"""

In [0]:
X1 = preprocess2(X1)

In [0]:
print(X1.shape)

In [0]:
shape_1 = 57
dynamic_scale_1 = o_data[1]/shape_1

In [0]:
BUFFER_SIZE = 6
BATCH_SIZE = 2

train_dataset = tf.data.Dataset.from_tensor_slices(X1).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [0]:
print(train_dataset)

In [0]:
# Generator Code
def generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(12*12*12*100, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    model.add(layers.Reshape((12,12,12,100)))
    assert model.output_shape == (None, 12,12,12,100)

    #Required (10,16,2) same:padding
    model.add(layers.Conv3DTranspose(512, (3, 3, 3), strides=(1, 1, 1), padding="same", use_bias=False))
    assert model.output_shape == (None, 12,12,12,512)
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    #Required (20,32,4) same:padding
    model.add(layers.Conv3DTranspose(256, (2, 2, 2), strides=(1, 1, 1), padding="same", use_bias=False))
    assert model.output_shape == (None, 12,12,12,256)
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())
    
    #Required (40,32,4) same:padding
    model.add(layers.Conv3DTranspose(128, (2, 2, 2), strides=(1, 1, 1), padding="same", use_bias=False))
    assert model.output_shape == (None, 12,12,12,128)
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    #Required (20,32,4) same:padding
    model.add(layers.Conv3DTranspose(64, (6, 6, 6), strides=(4, 4, 4), padding="same", use_bias=False))
    assert model.output_shape == (None, 48,48,48,64)
    model.add(layers.BatchNormalization())
    model.add(layers.ReLU())

    #Required (99,32,4) valid:padding    (3,2,1)
    model.add(layers.Conv3DTranspose(1, (2, 2, 2), strides=(1, 1, 1), padding="same", use_bias=False, activation="tanh"))
    assert model.output_shape == (None, 48,48,48,1) #final must be (48,48,48,1)
    return model

generator = generator_model()
noise = tf.random.normal([1, 100])
generated_image = generator(noise, training=False)

In [0]:
class separator(layers.Layer):
    def __init__(self):
        super(separator,self).__init__()

    def call(self, inputs, ratio):
        #l = inputs[:,:,:int(inputs.shape[2]/ratio)+1,:]
        #m = inputs[:,:,inputs.shape[2]-int(inputs.shape[2]/ratio)-1:,:]
        l,_ = tf.split(inputs,[int(inputs.shape[2]/ratio)+1,inputs.shape[2]-int(inputs.shape[2]/ratio)-1],2)
        _,m = tf.split(inputs,[int(inputs.shape[2]/ratio)-1,inputs.shape[2]-int(inputs.shape[2]/ratio)+1],2)
        return l,m

x_separator = separator()

In [0]:
# Discriminator Code
def discriminator_model():
    visible = layers.Input(shape=(48,48,48,1))

    split01, split02 = x_separator(visible, 67/57)
    conv11 = layers.Conv3D(32, kernel_size=5)(split01)
    conv11 = layers.LeakyReLU()(conv11)
    conv21 = layers.Conv3D(32, kernel_size=3)(split02)
    conv21 = layers.LeakyReLU()(conv21)
    conv22 = layers.Conv3D(32, kernel_size=3)(conv21)
    conv22 = layers.LeakyReLU()(conv22)

    merge1 = layers.concatenate([conv11,conv22],axis=2)

    split11, split12 = x_separator(merge1, 67/57)
    #split11, split12 = tf.split(merge1,[int(merge1.shape[2]/ratio),merge1.shape[2]-int(merge1.shape[2]/ratio)],2)
    conv12 = layers.Conv3D(32, kernel_size=5)(split11)
    conv12 = layers.LeakyReLU()(conv12)
    conv23 = layers.Conv3D(32, kernel_size=3)(split12)
    conv23 = layers.LeakyReLU()(conv23)
    conv24 = layers.Conv3D(32, kernel_size=3)(conv23)
    conv24 = layers.LeakyReLU()(conv24)

    merge2 = layers.concatenate([conv12,conv24],axis=2)

    split21, split22 = x_separator(merge2, 67/57)    
    #split21, split22 = tf.split(merge2,[int(merge2.shape[2]/ratio),merge2.shape[2]-int(merge2.shape[2]/ratio)],2)
    conv13 = layers.Conv3D(32, kernel_size=5)(split21)
    conv13 = layers.LeakyReLU()(conv13)
    conv25 = layers.Conv3D(32, kernel_size=3)(split22)
    conv25 = layers.LeakyReLU()(conv25)
    conv26 = layers.Conv3D(32, kernel_size=3)(conv25)
    conv26 = layers.LeakyReLU()(conv26)
    
    merge3 = layers.concatenate([conv13,conv26],axis=2)
    hidden1 = layers.Dense(10, activation='relu')(merge3)
    output = layers.Dense(1,activation='relu')(hidden1)


    model = tf.keras.models.Model(inputs=[visible], outputs=output)
    print(model.summary())

    return model

In [0]:
discriminator = discriminator_model()
decision = discriminator(generated_image)

In [0]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [0]:
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4,beta_1=0.5)#,beta_2=0.999,epsilon=1e-07)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=2e-4,beta_1=0.5)#,beta_2=0.999,epsilon=1e-07)

checkpoint_dir = '/content/drive/My Drive/Train_Checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer, discriminator_optimizer=discriminator_optimizer, generator=generator, discriminator=discriminator)
manager = tf.train.CheckpointManager(checkpoint, '/content/drive/My Drive/Train_Checkpoints', max_to_keep=3)

EPOCHS = 100
noise_dim = 100
num_examples_to_generate = 1
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [0]:
# Notice the use of `tf.function`
# This annotation causes the function to be "compiled".
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [0]:
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()

        for image_batch in dataset:
            train_step(image_batch)

        # Produce images for the GIF as we go
        display.clear_output(wait=True)
        generate(generator, epoch + 1, seed)

        # Save the model every 40 epochs
        if (epoch + 1) % 40 == 0:
            checkpoint.save(file_prefix = checkpoint_prefix)
        
        #checkpoint.restore(manager.latest_checkpoint)

        print ('Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))

    # Generate after the final epoch
    display.clear_output(wait=True)
    generate(generator, epochs, seed)

In [0]:
def original_shape(predictions):
    predictions = (predictions[0, :, :, :, 0] * (max_-min_)) + min_
    # Getting back in original form
    predictions = tf.reshape(predictions,[48,2304,1])
    predictions = tf.image.resize(predictions, [300,268])
    predictions = tf.reshape(predictions,[300,67,4])
    return predictions

In [0]:
def generate(model, epoch, test_input):
    # Notice `training` is set to False.
    # This is so all layers run in inference mode (batchnorm).
    predictions = model(test_input, training=False)
    predictions = original_shape(predictions)
    print(predictions)


In [0]:
train(train_dataset, EPOCHS)

In [0]:
def Save_animation(predictions)
    with open("/content/drive/My Drive/Result/test23.txt", 'w') as outfile:
        outfile.write('#Array shape: {0}\n'.format(predictions.shape))
        for data_slice in predictions:
            np.savetxt(outfile,data_slice)
            outfile.write('#New slice\n')

save_animation(predictions)